In [7]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score 
import numpy as np 


In [8]:

isu = pd.read_csv(r"..\Cases\Medical Cost Personal\insurance.csv")


dum_isu = pd.get_dummies(isu, drop_first=True)
X = dum_isu.drop('charges', axis=1)
y = dum_isu['charges']




In [9]:

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, 
                                                    random_state=24)
lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(r2_score(y_test, y_pred))


0.766542579868441


In [10]:

########## K-FOLD #############
kfold = KFold(n_splits=5, shuffle=True, 
              random_state=24)
lambdas = np.linspace(0.001, 100,40)
scores = []
for i in lambdas:
    lasso = Lasso(alpha=i)
    results = cross_val_score(lasso, X, y,
                              cv=kfold)
    scores.append(results.mean())

i_max = np.argmax(scores)
print("Best alpha =", lambdas[i_max])


Best alpha = 38.46215384615384


In [11]:
############################

from sklearn.model_selection import GridSearchCV
params = {'alpha':np.linspace(0.001, 100,40)}
gcv = GridSearchCV(lasso, param_grid=params,
                   cv=kfold)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

{'alpha': 38.46215384615384}
0.744531590596339


In [12]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(r2_score(y_test, y_pred))
from sklearn.model_selection import GridSearchCV
params = {'alpha':np.linspace(0.001, 100,40)}
gcv = GridSearchCV(ridge, param_grid=params,
                   cv=kfold)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

0.765876824167379
{'alpha': 0.001}
0.7442304540793144


In [13]:
elastic = ElasticNet()
from sklearn.model_selection import GridSearchCV

print(elastic.get_params())
params = {'alpha':np.linspace(0.001, 50,5),
          'l1_ratio':np.linspace(0.001,1, 10)}
gcv = GridSearchCV(elastic, param_grid=params,
                   cv=kfold, scoring='r2')
gcv.fit(X, y)
pd_cv = pd.DataFrame(gcv.cv_results_)
print(gcv.best_params_)
print(gcv.best_score_)

elastic.fit(X,y)
print(elastic.coef_)

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.5, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
{'alpha': 37.50025, 'l1_ratio': 1.0}
0.7445309871241849
[ 245.02779625  320.6540465   392.51968357  322.8450162  5833.36578447
  -99.18518978  124.90882135 -289.29980534]
